In [ ]:
# Notebook cell 1
import sys, pathlib
from tools.prompt_cache import cached_call

# add the project root so Python can see `personaltrainers/`
ROOT = pathlib.Path.cwd()          # change if you launch the notebook elsewhere
sys.path.append(str(ROOT))

# reload local modules each time you run the cell (handy while editing)
from crew import Personaltrainers
from tools.db_call import db_call


# 1: SQL FILTER GENERATOR
def sql_filter(client_request: str) -> str:
    inputs = {"user_request": client_request}
    result = Personaltrainers().crew().kickoff(inputs=inputs).raw
    result = result.replace("`", "")
    result = result.replace("\n", "")

    sql_filter = " ".join(result.split())

    return sql_filter

In [ ]:
import json
from pathlib import Path

# ---------------- Helper functions ---------------- #
def prompt_choice(question: str, options: list[str]) -> str:
    """Display a question with enumerated options and return the selected value."""
    print(question)
    for idx, opt in enumerate(options, start=1):
        print(f"  {idx}. {opt}")
    while True:
        choice = input("Select the corresponding number: ").strip()
        if choice.isdigit() and 1 <= int(choice) <= len(options):
            return options[int(choice) - 1]
        print("Invalid input, please try again.")

def prompt_yes_no(question: str) -> bool:
    """Return True if the user answers 'y/Y', False if 'n/N'."""
    while True:
        ans = input(f"{question} [y/n]: ").strip().lower()
        if ans in ("y", "n"):
            return ans == "y"
        print("Please answer with 'y' for yes or 'n' for no.")

def prompt_int_range(question: str, minimum: int, maximum: int) -> int:
    """Ask for an integer within [minimum, maximum] (inclusive)."""
    while True:
        ans = input(f"{question} ({minimum}-{maximum}): ").strip()
        if ans.isdigit() and minimum <= int(ans) <= maximum:
            return int(ans)
        print("Number out of range, please try again.")

# ---------------- Questionnaire ---------------- #
def run_questionnaire() -> dict:
    """Run all steps and return a dict representing the payload."""
    # 1. Main workout focus
    train_question = "What do you want to train today?"
    train_options  = ["Upper body", "Lower body", "Full body"]
    train_target = prompt_choice(train_question, train_options)

    # 2. Specific muscle focus (conditional)
    focus_muscle = prompt_yes_no("Do you want to put specific focus on certain muscles?")
    muscles = []
    if focus_muscle:
        # Map allowed muscles by primary focus
        muscle_map = {
            "Upper body": ["Chest", "Lats", "Shoulders", "Biceps", "Triceps"],
            "Lower body": ["Quads", "Hamstrings", "Glutes", "Calves"],
            "Full body" : ["Chest", "Back", "Shoulders", "Biceps", "Triceps",
                           "Quads", "Hamstrings", "Glutes", "Calves"],
        }
        available = muscle_map[train_target]
        print("Select the muscle groups (comma-separated):")
        for idx, mus in enumerate(available, start=1):
            print(f"  {idx}. {mus}")
        while True:
            raw = input("Enter the corresponding numbers: ").replace(" ", "")
            indices = [s for s in raw.split(",") if s.isdigit()]
            if all(1 <= int(i) <= len(available) for i in indices):
                muscles = [available[int(i)-1] for i in indices]
                break
            print("Invalid selection, please try again.")

    # 3. Time available
    minutes = prompt_int_range("How much time do you have available (in minutes)?", 45, 180)

    # 4. Training location
    location = prompt_choice("Where are you training?", ["Gym", "Park", "Home"])

    # Assemble payload
    payload = {
        "train_target": train_target,
        "focus_muscle": focus_muscle,
        "muscles": muscles,               # empty list if none
        "duration_minutes": minutes,
        "location": location
    }
    return payload

# ---------------- Main entry point ---------------- #
answers = run_questionnaire()
file_path = Path("workout_payload.json")
with file_path.open("w", encoding="utf-8") as fp:
    json.dump(answers, fp, ensure_ascii=False, indent=2)
print("\n--- Answers saved! ---")
print(json.dumps(answers, ensure_ascii=False, indent=2))
print(f"\nJSON file created: {file_path.resolve()}")

What do you want to train today?
  1. Upper body
  2. Lower body
  3. Full body
Select the muscle groups (comma-separated):
  1. Chest
  2. Lats
  3. Shoulders
  4. Biceps
  5. Triceps
Where are you training?
  1. Gym
  2. Park
  3. Home

--- Answers saved! ---
{
  "train_target": "Upper body",
  "focus_muscle": true,
  "muscles": [
    "Chest",
    "Shoulders"
  ],
  "duration_minutes": 120,
  "location": "Gym"
}

JSON file created: C:\Users\Emanuele\Desktop\LazyTrainer\personaltrainers\src\personaltrainers\workout_payload.json


In [66]:
with file_path.open("r", encoding="utf-8") as fp:
    answers = json.load(fp)

answers

{'train_target': 'Upper body',
 'focus_muscle': True,
 'muscles': ['Chest', 'Shoulders'],
 'duration_minutes': 120,
 'location': 'Gym'}

In [94]:
prompt = "Select exercises to build an upper body routine with focus on shoulders"

In [84]:
prompt = f"Select the exercises using these informations: {answers.get('train_target')}, {answers.get('muscles')}"
prompt = prompt.replace("'", "")
prompt = prompt.replace("[", "")
prompt = prompt.replace("]", "")
prompt

'Select the exercises using these informations: Upper body, Chest, Shoulders'

In [ ]:
sql_filter = cached_call(prompt, sql_filter)
exercises_df = db_call(sql_filter)

TypeError: 'str' object is not callable

In [58]:
exercises_df.to_dict(orient="records")

[{'id': 1,
  'name': 'horizontal bench press',
  'movement_pattern': 'horizontal push',
  'movement_type': 'compound',
  'body_region': 'upper body',
  'load_class': 'free-weight',
  'muscles': 'chest, shoulders, triceps',
  'equipments': 'barbell, dumbbells, smith machine'},
 {'id': 2,
  'name': 'incline bench press',
  'movement_pattern': 'horizontal push',
  'movement_type': 'compound',
  'body_region': 'upper body',
  'load_class': 'free-weight',
  'muscles': 'chest, shoulders, triceps',
  'equipments': 'barbell, dumbbells, smith machine'},
 {'id': 3,
  'name': 'parallel bar dips',
  'movement_pattern': 'horizontal push',
  'movement_type': 'compound',
  'body_region': 'upper body',
  'load_class': 'bodyweight',
  'muscles': 'chest, shoulders, triceps',
  'equipments': 'gymnastics rings, parallel bars'},
 {'id': 4,
  'name': 'push ups',
  'movement_pattern': 'horizontal push',
  'movement_type': 'compound',
  'body_region': 'upper body',
  'load_class': 'bodyweight',
  'muscles': '